<H1> Разработка инструмента для прогнозирования популярности постов в социальных сетях с применением методов машинного обучения </H1>

Проект содержит следующие этапы:
* Сбор БД и визуализация данных
* Токенизация, лемматизация и стемминг данных
* Векторизация
* **Классификация**
* Подбор гиперпараметров и демонстрация итоговых результатов

Для корректной работы кода необходимо загрузить файлы "ITMO_2.csv",  "ITMO_2_vectorized_tf_idf
.npz", полученные в рамках предыдущего шага проекта.

# **Разделение на обучающую и тестовую выборки**

In [ ]:
import numpy as np

# Загрузка данных из файла .npz
data = np.load('ITMO_2_vectorized_tf_idf.npz', allow_pickle=True)

# Вывод списка переменных в файле
print("Переменные в файле:")
for variable_name in data.files:
    print(variable_name)

Переменные в файле:
X
feature_names


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Загрузка данных из файла .npz
data_npz = np.load('ITMO_2_vectorized_tf_idf.npz', allow_pickle=True)

# Извлечение переменной 'data' из файла .npz
X_data = data_npz['X']

# Загрузка целевой переменной из файла ITMO_2.csv
y_data = pd.read_csv('ITMO_2.csv')['popular']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [ ]:
# размер обучающей и тестовой выборки
print(len(X_train))
print(len(X_test))

12501
3126


In [ ]:
# метрики качества
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# **Наивный Байес**
Наивный Байес рассматривает классификацию как вероятностную задачу - он вычисляет вероятность принадлежности объекта к каждому классу и выбирает тот класс, для которого вероятность максимальна.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf_nb = MultinomialNB(
    # alpha (default=1.0) - параметр регуляризации
    # Более высокие значения делают модель более устойчивой к переобучению
        alpha = 1.1
)

In [ ]:
clf_nb = clf_nb.fit(X_train, y_train)
y_predict_nb = clf_nb.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_nb))
print('Precision: ', precision_score(y_test, y_predict_nb, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_nb, average='macro'))
print('F1: ', f1_score(y_test, y_predict_nb, average='macro'))

Accuracy:  0.8195777351247601
Precision:  0.5766890810118476
Recall:  0.5004979316949983
F1:  0.4521712867386142


# **Случайный лес**
Случайный лес использует ансамблевое обучение, сочетающее в себе множество независимыо обученных решающих деревьев.



In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_rf =  RandomForestClassifier(
        n_estimators = 200, # количество деревьев в ансамбле (default=100)
        # какая функция используется для измерения качества разбиения деревьев
        criterion = 'gini', # gini - Индекс Джини, entropy - Энтропия, log_loss - логарифмическая потеря, default gini
        max_depth = 200, # максимальная глубина дерева, int, default=None
        # уменьшение дисбаланса классов
        # dict - пользовательские настройки
        # balanced - автоматическая балансировка классов
        # None - по умолчанию классы имеют одинаковые веса
        class_weight = 'balanced', # default=None
        random_state = 42 # контроль воспроизводимости результатов при использовании случайности
)

In [ ]:
clf_rf = clf_rf.fit(X_train, y_train)
y_predict_rf = clf_rf.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_rf))
print('Precision: ', precision_score(y_test, y_predict_rf, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_rf, average='macro'))
print('F1: ', f1_score(y_test, y_predict_rf, average='macro'))

Accuracy:  0.8422904670505438
Precision:  0.8157129292091418
Recall:  0.5801853678076244
F1:  0.5958671997000048


# **Деревья решений**
Решающее дерево строится рекурсивно, последовательно разделяя данные на подмножества на основе наиболее информативных признаков. Каждый внутренний узел дерева содержит условие для разделения данных, а листовые узлы представляют собой предсказания классов.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf_dt =  DecisionTreeClassifier(
        # определяет метрику, по которой будет происходить разделение
        # 'gini' - индекс Джини (по умолчанию)
        # 'entropy' - энтропия
        # 'log_loss' - логарифмические потери
        criterion = 'entropy',
        # стратегия выбора признака
        # 'best' - лучшией признак значения (по умолчанию)
        # 'random' - выбирает случайный признак
        splitter = 'best',
        max_depth = 200, # максимальная глубина дерева, int, default=None
        # учет дисбаланса классов при обучении модели
        class_weight = 'balanced', #dict or ‘balanced’, default=None
        random_state = 42 # задает начальное значение генератора для воспроизводимости результатов
)

In [ ]:
clf_dt = clf_dt.fit(X_train, y_train)
y_predict_dt = clf_dt.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_dt))
print('Precision: ', precision_score(y_test, y_predict_dt, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_dt, average='macro'))
print('F1: ', f1_score(y_test, y_predict_dt, average='macro'))

Accuracy:  0.77191298784389
Precision:  0.6304078219445991
Recall:  0.6467633746809529
F1:  0.6371655441422883


# **K-ближайших соседей**

k-ближайших соседей строится на том, что для классификации нового объекта используются его ближайшие соседи в признаковом пространстве (расстояние до них).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf_KNN = KNeighborsClassifier(
        n_neighbors = 3, # int, default=5, кол-во ближайших соседей
        # способ взвешивания вклада соседей в определение класса
        # 'uniform' - все соседи имеют одинаковый вес
        # 'distance' - вес соседа прямо пропорционален его расстоянию до объекта
        weights = 'distance', # default=’uniform’
        # метрика расстояния, используемая для поиска
        # 'euclidean' - Евклидово расстояние
        # 'manhattan' - Манхэттенское расстояние
        # 'chebyshev' - расстояние Чебышева
        # 'cosine' - косинусное сходство
        # 'minkowski' - расстояние Минковского
        metric = 'euclidean', # default=’minkowski’
)

In [ ]:
clf_KNN = clf_KNN.fit(X_train, y_train)
y_predict_KNN = clf_KNN.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_KNN))
print('Precision: ', precision_score(y_test, y_predict_KNN, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_KNN, average='macro'))
print('F1: ', f1_score(y_test, y_predict_KNN, average='macro'))

Accuracy:  0.8224568138195777
Precision:  0.7526358950328023
Recall:  0.5119559048046077
F1:  0.4766733178591305


# **Метод опорных векторов**

Основная идея метода опорных векторов (Support Vector Machines, SVM) заключается в поиске оптимальной гиперплоскости, которая максимально разделяет классы в многомерном пространстве признаков.

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
clf_lsvc = LinearSVC(
        # тип регуляризации
        # l1 - сумма абсолютных значений весов
        # l2 - сумма квадратов весов
        penalty = 'l2', # default=’l2’
        # коэффициент регуляризации; чем меньше, тем больше регуляризация
        C = 0.2, #float, default=1.0
        # способ учета дисбаланса классов
        class_weight = 'balanced', # default=None
        # начальное состояние генератора для воспроизводимости результата
        random_state = 42,
        # максимальное число итераций оптимизации
        max_iter = 2000 # int, default=1000
)

In [ ]:
clf_lsvc = clf_lsvc.fit(X_train, y_train)
y_predict_lsvc = clf_lsvc.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_lsvc))
print('Precision: ', precision_score(y_test, y_predict_lsvc, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_lsvc, average='macro'))
print('F1: ', f1_score(y_test, y_predict_lsvc, average='macro'))

Accuracy:  0.8096609085092771
Precision:  0.7013151435289254
Recall:  0.759182283195273
F1:  0.7205847204547752


# **Логистическая регрессия**

Основная идея логистической регрессии заключается в моделировании вероятности принадлежности объекта к определенному классу с использованием сигмоидальной функции активации.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_lr = LogisticRegression(
        # тип регуляризации
        # l1 - сумма абсолютных значений весов
        # l2 - сумма квадратов весов
        penalty = 'l2', # default=’l2’
        # коэффициент регуляризации; чем меньше, тем больше регуляризация
        C = 0.2, # float, default=1.0
        # способ учета дисбаланса классов
        class_weight = 'balanced', # default=None
        # начальное состояние генератора для воспроизводимости результата
        random_state = 42,
        # максимальное число итераций оптимизации
        max_iter = 100, # int, default=100
        # кол-во используемых ядер процессора
        n_jobs = -1, # int, default=None
)

In [ ]:
clf_lr = clf_lr.fit(X_train, y_train)
y_predict_lr = clf_lr.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_predict_lr))
print('Precision: ', precision_score(y_test, y_predict_lr, average='macro'))
print('Recall: ', recall_score(y_test, y_predict_lr, average='macro'))
print('F1: ', f1_score(y_test, y_predict_lr, average='macro'))

Accuracy:  0.7680742162507997
Precision:  0.672508882567319
Recall:  0.7511467675327744
F1:  0.6878155712995018


| Модель | Accuracy | Precision | Recall | F1 |
|--------|----------|-----------|--------|----|
| Наивный Байес | 0.8195777351247601 | 0.5766890810118476 | 0.5004979316949983 | 0.4521712867386142 |
| **Случайный лес** | 0.8422904670505438 | 0.8157129292091418 | 0.5801853678076244 | 0.5958671997000048 |
| Деревья решений | 0.77191298784389 | 0.6304078219445991 | 0.6467633746809529 | 0.6371655441422883 |
| K-ближайших соседей | 0.8224568138195777 | 0.7526358950328023 | 0.5119559048046077 | 0.4766733178591305 |
| Метод опорных векторов | 0.8096609085092771 | 0.7013151435289254 | 0.759182283195273 | 0.7205847204547752 |
| Логистическая регрессия | 0.7680742162507997 | 0.672508882567319 | 0.7511467675327744 | 0.6878155712995018 |

Исходя из полученных метрик качества, можно сделать следующие выводы:

1. Случайный лес (Random Forest) показывает лучшие результаты по большинству метрик:
   - Accuracy: 0.8422904670505438 (самая высокая точность)
   - Precision: 0.8157129292091418 (самая высокая точность прогнозирования положительных классов)
   - F1-score: 0.5958671997000048 (самый высокий гармонический показатель точности и полноты)

2. Деревья решений (Decision Trees) также показывают неплохие результаты, особенно по полноте (Recall: 0.6467633746809529) и F1-score (0.6371655441422883).

3. Метод опорных векторов (SVM) демонстрирует хорошие результаты по полноте (Recall: 0.759182283195273) и F1-score (0.7205847204547752).

4. Наивный Байес (Naive Bayes) и K-ближайших соседей (KNN) показывают более низкие результаты по сравнению со случайным лесом, деревьями решений и SVM.

5. Логистическая регрессия занимает промежуточное положение между более простыми моделями (Naive Bayes, KNN) и более сложными (Random Forest, Decision Trees, SVM).<br>
Победитель по метрикам - **Случайный лес**